# How does targeted cash assistance affect incentives to work?

In [23]:
try:
    import openfisca_us
except ImportError as e:
    !pip install openfisca-us

In [24]:
from openfisca_us import IndividualSim, Microsimulation

In [3]:
from openfisca_us.model_api import *
from openfisca_us.tools.baseline_variables import baseline_variables

# Set numbers from proposal
POV_THRESHOLD = 2
MONTHLY_AMOUNT = 500
FULL_DURATION_MONTHS = 18


class cambridge_cash_18mo(Variable):
    value_type = float
    entity = SPMUnit
    definition_period = YEAR

    def formula(spm_unit, period, parameters):
        pov_ratio = spm_unit("school_meal_fpg_ratio", period)
        income_eligible = pov_ratio < POV_THRESHOLD
        has_children = add(spm_unit, period, ["is_child"]) > 0
        return income_eligible * has_children * MONTHLY_AMOUNT * FULL_DURATION_MONTHS


class spm_unit_net_income(baseline_variables["spm_unit_net_income"]):
    def formula(spm_unit, period, parameters):
        original_net_income = baseline_variables[
            "spm_unit_net_income"
        ].formula(spm_unit, period, parameters)
        return original_net_income + spm_unit("cambridge_cash_18mo", period)


class add_cambridge_cash_18mo(Reform):
    def apply(self):
        self.update_variable(cambridge_cash_18mo)
        self.update_variable(spm_unit_net_income)

Run IndividualSim.

In [4]:
def family_sim(reform=None):
    if reform is None:
        sim = IndividualSim(year=2022)
    else:
        sim = IndividualSim(reform, year=2022)
    sim.add_person(name="p1", age=30)
    sim.add_person(name="p2", age=30)
    sim.add_person(name="c1", age=10)
    sim.add_person(name="c2", age=10)
    sim.add_spm_unit(name="spm_unit", members=["p1", "p2", "c1", "c2"],
                     broadband_cost=600, housing_cost=24_000)
    sim.add_household(name="household", members=["p1", "p2", "c1", "c2"],
                      state_code_str="MA")
    sim.vary("employment_income", max=100_000)
    return sim


baseline = family_sim()
reformed = family_sim(add_cambridge_cash_18mo)

Marginal tax rates

In [5]:
def make_df(sim, scenario):
    return pd.DataFrame(dict(
        employment_income = sim.calc("employment_income")[0],
        net_income = sim.calc("spm_unit_net_income")[0],
        mtr = 1 - sim.deriv("spm_unit_net_income", "employment_income",
                            wrt_target="p1"),
        scenario=scenario
        )
    )

import plotly.express as px
import pandas as pd

df = pd.concat([make_df(baseline, "Baseline"),
                make_df(reformed, "Cambridge cash")])

In [6]:
def gap(sim):
    employment_income = sim.calc("employment_income")[0]
    net_income = sim.calc("spm_unit_net_income")[0]
    diffs = np.diff(net_income, append=np.inf)
    cliffs = np.where(diffs < 0)[0]
    l = []
    for cliff in cliffs:
        employment_income_before_cliff = employment_income[cliff]
        net_income_before_cliff = net_income[cliff]
        ix_first_exceed_cliff = np.argmax(net_income > net_income_before_cliff)
        employment_income_after_cliff = employment_income[ix_first_exceed_cliff]
        l += [[employment_income_before_cliff, employment_income_after_cliff]]
    return l

gap(reformed)

[[55400.0, 84600.0], [55500.0, 43500.0]]

In [26]:
from plotly import graph_objects as go

LABELS = dict(
    employment_income="Employment income",
    net_income="Net income",
    mtr="Marginal tax rate",
    scenario="Scenario"
    )

COLORS = {"Baseline": "gray", "Cambridge cash": "blue"}

# fig = go.Figure()
# fig.add_trace(
fig = px.line(
    df[df.scenario == "Baseline"],
    "employment_income",
    "net_income",
    color="scenario",
    labels=LABELS,
    color_discrete_map=COLORS
)

baseline_gap_bounds = gap(baseline)
print(baseline_gap_bounds)

fig.update_layout(plot_bgcolor='white')

ymax = df[df.scenario == "Baseline"].net_income.max()

for gap_bound in baseline_gap_bounds:
    start = gap_bound[0]
    end = gap_bound[1]
    fig.add_trace(
        go.Scatter(
            x=[start, start, end, end, start], 
            y=[0, ymax, ymax, 0, 0], 
            fill="toself",
            mode='lines',
            fillcolor="lightgray",
            name='',
            text="This household is worse off earning between " +
            "${:,}".format(int(start)) + " and ${:,}".format(int(end)),
            opacity=0.2,
            line_width=0,
            showlegend=False,
        )
    )
fig.show()
fig.write_html("output/baseline.html")

[[55500.0, 70900.0]]


In [ ]:
from plotly import graph_objects as go

LABELS = dict(
    employment_income="Employment income",
    net_income="Net income",
    mtr="Marginal tax rate",
    scenario="Scenario"
    )

COLORS = {"Baseline": "gray", "Cambridge cash": "blue"}

# fig = go.Figure()
# fig.add_trace(
fig = px.line(
    df[df.scenario == "Baseline"],
    "employment_income",
    "net_income",
    color="scenario",
    labels=LABELS,
    color_discrete_map=COLORS
)

baseline_gap_bounds = gap(baseline)
print(baseline_gap_bounds)

fig.update_layout(plot_bgcolor='white')

ymax = df[df.scenario == "Baseline"].net_income.max()

for gap_bound in baseline_gap_bounds:
    start = gap_bound[0]
    end = gap_bound[1]
    fig.add_trace(
        go.Scatter(
            x=[start, start, end, end, start], 
            y=[0, ymax, ymax, 0, 0], 
            fill="toself",
            mode='lines',
            fillcolor="lightgray",
            name='',
            text="This household is worse off earning between " +
            "${:,}".format(int(start)) + " and ${:,}".format(int(end)),
            opacity=0.2,
            line_width=0,
            showlegend=False,
        )
    )
fig.show()
fig.write_html("output/baseline.html")

Now add child allowance.

In [28]:
fig = px.line(df, "employment_income", "net_income", color="scenario",
              title="Family with two parents and two children", labels=LABELS,
              color_discrete_map=COLORS)

reformed_gap_bounds = gap(reformed)

fig.update_layout(plot_bgcolor='white')

for gap_bound in baseline_gap_bounds:
    fig.add_vrect(
        x0=gap_bound[0],
        x1=gap_bound[1],
        fillcolor="gray",
        opacity=0.05,
        line_width=0,
    )

for gap_bound in reformed_gap_bounds:
    if gap_bound[1] > gap_bound[0]:  # Bug.
        fig.add_vrect(
            x0=gap_bound[0],
            x1=gap_bound[1],
            fillcolor="blue",
            opacity=0.05,
            line_width=0,
        )
fig.show()
fig.write_html("output/cambridge-cash.html")

Microsim: 2020 is latest year.

In [10]:
# Redefine for 12 instead of 18 months.
class cambridge_cash_12mo(Variable):
    value_type = float
    entity = SPMUnit
    definition_period = YEAR

    def formula(spm_unit, period, parameters):
        pov_ratio = spm_unit("school_meal_fpg_ratio", period)
        income_eligible = pov_ratio < POV_THRESHOLD
        has_children = add(spm_unit, period, ["is_child"]) > 0
        return income_eligible * has_children * MONTHLY_AMOUNT * 12


class spm_unit_net_income(baseline_variables["spm_unit_net_income"]):
    def formula(spm_unit, period, parameters):
        original_net_income = baseline_variables[
            "spm_unit_net_income"
        ].formula(spm_unit, period, parameters)
        # vehicle_payment = add(spm_unit, period, ["per_vehicle_payment"])
        return original_net_income + spm_unit("cambridge_cash_12mo", period)


class add_cambridge_cash_12mo(Reform):
    def apply(self):
        self.update_variable(cambridge_cash_12mo)
        self.update_variable(spm_unit_net_income)

In [11]:
mbaseline = Microsimulation(year=2020)
mreformed = Microsimulation(add_cambridge_cash_12mo, year=2020)

In [12]:
# Filter to MA.
person_weights = mbaseline.calc("person_weight")
spm_unit_weights = mbaseline.calc("spm_unit_weight")
household_weights = mbaseline.calc("household_weight")
state_code = mbaseline.calc("state_code_str", map_to="person")
state_code_household = mbaseline.calc("state_code_str")

for m in [mbaseline, mreformed]:
    m.set_input("person_weight", m.year,
                person_weights * (state_code == "MA"))
    m.set_input("household_weight", m.year,
                household_weights * (state_code_household == "MA"))

In [13]:
mbaseline.calc("spm_unit_is_in_spm_poverty", map_to="person").mean()

0.08095032984237577

In [14]:
mreformed.calc("spm_unit_is_in_spm_poverty", map_to="person").mean()

0.06916880202736501

In [15]:
def get_metrics(sim, name):
    pov = sim.calc("spm_unit_is_in_spm_poverty", map_to="person")
    deep_pov = sim.calc("spm_unit_is_in_deep_spm_poverty", map_to="person")
    child = sim.calc("is_child")
    return pd.Series(dict(
        poverty=pov.mean(),
        child_poverty=(pov * child).sum() / child.sum(),
        deep_poverty=deep_pov.mean(),
        deep_child_poverty=(deep_pov * child).sum() / child.sum(),
    ), name=name)

In [16]:
poverty_impact = pd.DataFrame([
    get_metrics(mbaseline, "Baseline"),
    get_metrics(mreformed, "Cambridge cash")])
poverty_impact.loc["Diff"] = poverty_impact.loc["Cambridge cash"] / poverty_impact.loc["Baseline"] - 1
poverty_impact

,poverty,child_poverty,deep_poverty,deep_child_poverty
Baseline,0.080950,0.081025,0.023105,0.012157
Cambridge cash,0.069169,0.047354,0.019330,0.000000
Diff,-0.145540,-0.415559,-0.163392,-1.000000


## Child allowance comparison

In [17]:
budget = mreformed.calc("cambridge_cash_12mo", map_to="household").sum()
budget

1185175500.7324219

In [18]:
kids = mreformed.calc("is_child").sum()
kids

1350342.9387207031

In [19]:
child_allowance_amount = budget / kids
child_allowance_amount / 12
CHILD_ALLOWANCE = 75

73.14040176683076

In [20]:
def make_child_allowance(amount)

class child_allowance(Variable):
    value_type = float
    entity = Person
    definition_period = YEAR

    def formula(person, period, parameters):
        # Manually try the budget-neutral child allowance or $200/mo.
        return 200 * 12 * person("is_child", period)
        return child_allowance_amount * person("is_child", period)


class spm_unit_net_income(baseline_variables["spm_unit_net_income"]):
    def formula(spm_unit, period, parameters):
        original_net_income = baseline_variables[
            "spm_unit_net_income"
        ].formula(spm_unit, period, parameters)
        return original_net_income + add(spm_unit, period, ["child_allowance"])


class add_child_allowance(Reform):
    def apply(self):
        self.update_variable(child_allowance)
        self.update_variable(spm_unit_net_income)

In [21]:
mchildallowance = Microsimulation(add_child_allowance, year=2020)

for m in [mchildallowance]:
    m.set_input("person_weight", m.year,
                person_weights * (state_code == "MA"))
    m.set_input("household_weight", m.year,
                household_weights * (state_code_household == "MA"))

In [22]:
poverty_impact = pd.DataFrame([
    get_metrics(mbaseline, "Baseline"),
    get_metrics(mchildallowance, "Child allowance")])
poverty_impact.loc["Diff"] = poverty_impact.loc["Child allowance"] / poverty_impact.loc["Baseline"] - 1
poverty_impact

,poverty,child_poverty,deep_poverty,deep_child_poverty
Baseline,0.080950,0.081025,0.023105,0.012157
Child allowance,0.068360,0.044987,0.019330,0.000000
Diff,-0.155526,-0.444776,-0.163392,-1.000000
